# Setting up the PyTorch Lightning model

In [1]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
from torchmetrics import Accuracy


class MultiLayerPerceptron(pl.LightningModule):
    def __init__(self, image_shape=(1, 28, 28), hidden_units=(32, 16)):
        super().__init__()

        # new PL attributes:
        self.train_acc = Accuracy()
        self.valid_acc = Accuracy()
        self.test_acc = Accuracy()

        # Model similar to previous section:
        input_size = image_shape[0] * image_shape[1] * image_shape[2]
        all_layers = [nn.Flatten()]
        for unit in hidden_units:
            layer = nn.Linear(input_size, unit)
            all_layers.append(layer)
            all_layers.append(nn.ReLU())
            input_size = unit

        all_layers.append(nn.Linear(hidden_units[-1], 10))
        self.model = nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.train_acc.update(preds, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def training_epoch_end(self, outs):
        self.log('Train_acc', self.train_acc.compute())

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.valid_acc.update(preds, y)
        self.log('Valid_loss', loss, prog_bar=True)
        self.log('Valid_acc', self.valid_acc.compute(), prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y)
        self.log('Test_loss', loss, prog_bar=True)
        self.log('Test_acc', self.test_acc.compute(), prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer


# Setting up the data loaders for Lightning

In [2]:
import re
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms


class MnistDataModule(pl.LightningDataModule):
    def __init__(self, data_path='./'):
        super().__init__()
        self.data_path = data_path
        self.transform = transforms.Compose([transforms.ToTensor()])

    def prepare_data(self):
        MNIST(root=self.data_path, download=False)

    def setup(self, stage=None):
        # stage is either 'fit', 'validate', 'test', or 'predict'
        # here note relevant
        mnist_all = MNIST(
            root=self.data_path,
            train=True,
            transform=self.transform,
            download=False
        )
        self.train, self.val = random_split(
            mnist_all, [55000, 5000], generator=torch.Generator().manual_seed(1))
        self.test = MNIST(
            root=self.data_path,
            train=False,
            transform=self.transform,
            download=False
        )

    def train_dataloader(self):
        return DataLoader(self.train, batch_size=64, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=64, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=64, num_workers=4)


In [3]:
torch.manual_seed(1)
mnist_dm = MnistDataModule()


# Training the model using the PyTorch Lightning Trainer class

In [4]:
mnist_clf = MultiLayerPerceptron()
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model=mnist_clf, datamodule=mnist_dm)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: f:\courses trainig folders\Python Machine Learning\Python-Machine-Learning-Notebooks\The Mechanics of PyTorch\lightning_logs

  | Name      | Type       | Params
-----------------------------------------
0 | train_acc | Accuracy   | 0     
1 | valid_acc | Accuracy   | 0     
2 | test_acc  | Accuracy   | 0     
3 | model     | Sequential | 25.8 K
-----------------------------------------
25.8 K    Trainable params
0         Non-trainable params
25.8 K    Total params
0.103     Total estimated model params size (MB)


Epoch 9: 100%|██████████| 939/939 [00:31<00:00, 29.97it/s, loss=0.1, v_num=0, train_loss=0.260, Valid_loss=0.166, Valid_acc=0.936]     

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 939/939 [00:31<00:00, 29.96it/s, loss=0.1, v_num=0, train_loss=0.260, Valid_loss=0.166, Valid_acc=0.936]


# Evaluating the model using TensorBoard

In [5]:
%load_ext tensorboard
%tensorboard - -logdir lightning_logs/


Reusing TensorBoard on port 6006 (pid 7164), started 3:27:51 ago. (Use '!kill 7164' to kill it.)

In [6]:
# Lightning allows us to load a trained model and train it for additional epochs
trainer = pl.Trainer(
    max_epochs=15, resume_from_checkpoint='./lightning_logs/version_0/checkpoints/epoch=9-step=8600.ckpt')
trainer.fit(model=mnist_clf, datamodule=mnist_dm)


c:\Users\promar\anaconda3\lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:52: LightningDeprecationWarning: Setting `Trainer(resume_from_checkpoint=)` is deprecated in v1.5 and will be removed in v1.7. Please pass `Trainer.fit(ckpt_path=)` directly instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\promar\anaconda3\lib\site-packages\pytorch_lightning\trainer\trainer.py:731: LightningDeprecationWarning: `trainer.resume_from_checkpoint` is deprecated in v1.5 and will be removed in v2.0. Specify the fit checkpoint path with `trainer.fit(ckpt_path=)` instead.
  ckpt_path = ckpt_path or self.resume_from_checkpoint
Restoring states from the checkpoint path at ./lightning_logs/version_0/checkpoints/epoch=9-step=8600.ckpt
c:\Users\promar\anaconda3\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:345: UserWarnin

Epoch 14: 100%|██████████| 939/939 [00:31<00:00, 29.42it/s, loss=0.0762, v_num=1, train_loss=0.158, Valid_loss=0.168, Valid_acc=0.941]  

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 939/939 [00:31<00:00, 29.40it/s, loss=0.0762, v_num=1, train_loss=0.158, Valid_loss=0.168, Valid_acc=0.941]


In [7]:
# evaluate the model on the test set
trainer.test(model=mnist_clf, datamodule=mnist_dm)


Testing DataLoader 0: 100%|██████████| 157/157 [00:02<00:00, 65.56it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Test_acc            0.9517738223075867
        Test_loss           0.14718946814537048
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'Test_loss': 0.14718946814537048, 'Test_acc': 0.9517738223075867}]

In [8]:
# PyTorch Lightning also saves the model automatically
# model = MultiLayerPerceptron.load_from_checkpoint("./lightning_logs/version_0/checkpoints/epoch=9-step=8600.ckpt")
